In [3]:
from carts_poles import CartsPolesEnv
import torch
import torch.nn
import numpy as np
import random
import gym
from collections import namedtuple
from collections import deque
from typing import List, Tuple
import matplotlib.pyplot as plt
import time

In [15]:
# Hyper parms!!
global FLAGS

FLAGS = {
    "gamma" : 0.99,
    
    "n_episode" : 1000,

    "batch_size" : 64,

    "hidden_dim" : 12,

    "capacity" : 50000,

    "max_episode" : 50,

    "min_eps" : 0.01
}

In [16]:
class DQN(torch.nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int) -> None:
        """DQN Network
        Args:
            input_dim (int): `state` dimension.
                `state` is 2-D tensor of shape (n, input_dim)
            output_dim (int): Number of actions.
                Q_value is 2-D tensor of shape (n, output_dim)
            hidden_dim (int): Hidden dimension in fc layer
        """
        super(DQN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.PReLU()
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.PReLU()
        )

        self.final = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Returns a Q_value
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, input_dim)
        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, output_dim)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.final(x)

        return x

class ReplayMemory(object):

    def __init__(self, capacity: int) -> None:
        """Replay memory class
        Args:
            capacity (int): Max size of this memory
        """
        self.capacity = capacity
        self.cursor = 0
        self.memory = []

    def push(self,
             state: np.ndarray,
             action: int,
             reward: int,
             next_state: np.ndarray,
             done: bool) -> None:
        """Creates `Transition` and insert
        Args:
            state (np.ndarray): 1-D tensor of shape (input_dim,)
            action (int): action index (0 <= action < output_dim)
            reward (int): reward value
            next_state (np.ndarray): 1-D tensor of shape (input_dim,)
            done (bool): whether this state was last step
        """
        if len(self) < self.capacity:
            self.memory.append(None)

        self.memory[self.cursor] = Transition(state,
                                              action, reward, next_state, done)
        self.cursor = (self.cursor + 1) % self.capacity

    def pop(self, batch_size: int) -> List[Transition]:
        """Returns a minibatch of `Transition` randomly
        Args:
            batch_size (int): Size of mini-bach
        Returns:
            List[Transition]: Minibatch of `Transition`
        """
        return random.sample(self.memory, batch_size)

    def __len__(self) -> int:
        """Returns the length """
        return len(self.memory)


class Agent(object):

    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int) -> None:
        """Agent class that choose action and train
        Args:
            input_dim (int): input dimension
            output_dim (int): output dimension
            hidden_dim (int): hidden dimension
        """
        self.dqn = DQN(input_dim, output_dim, hidden_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.loss_fn = torch.nn.MSELoss()
        self.optim = torch.optim.Adam(self.dqn.parameters())

    def _to_variable(self, x: np.ndarray) -> torch.Tensor:
        """torch.Variable syntax helper
        Args:
            x (np.ndarray): 2-D tensor of shape (n, input_dim)
        Returns:
            torch.Tensor: torch variable
        """
        return torch.autograd.Variable(torch.Tensor(x))

    def get_action(self, states: np.ndarray, eps: float) -> int:
        """Returns an action
        Args:
            states (np.ndarray): 2-D tensor of shape (n, input_dim)
            eps (float): 𝜺-greedy for exploration
        Returns:
            int: action index
        """
        if np.random.rand() < eps:
            return np.random.choice(self.output_dim)
        else:
            self.dqn.train(mode=False)
            scores = self.get_Q(states)
            _, argmax = torch.max(scores.data, 1)
            return int(argmax.numpy())

    def get_Q(self, states: np.ndarray) -> torch.FloatTensor:
        """Returns `Q-value`
        Args:
            states (np.ndarray): 2-D Tensor of shape (n, input_dim)
        Returns:
            torch.FloatTensor: 2-D Tensor of shape (n, output_dim)
        """
        states = self._to_variable(states.reshape(-1, self.input_dim))
        self.dqn.train(mode=False)
        return self.dqn(states)

    def train(self, Q_pred: torch.FloatTensor, Q_true: torch.FloatTensor) -> float:
        """Computes `loss` and backpropagation
        Args:
            Q_pred (torch.FloatTensor): Predicted value by the network,
                2-D Tensor of shape(n, output_dim)
            Q_true (torch.FloatTensor): Target value obtained from the game,
                2-D Tensor of shape(n, output_dim)
        Returns:
            float: loss value
        """
        self.dqn.train(mode=True)
        self.optim.zero_grad()
        loss = self.loss_fn(Q_pred, Q_true)
        loss.backward()
        self.optim.step()

        return loss


def train_helper(agent: Agent, minibatch: List[Transition], gamma: float) -> float:
    """Prepare minibatch and train them
    Args:
        agent (Agent): Agent has `train(Q_pred, Q_true)` method
        minibatch (List[Transition]): Minibatch of `Transition`
        gamma (float): Discount rate of Q_target
    Returns:
        float: Loss value
    """
    states = np.vstack([x.state for x in minibatch])
    actions = np.array([x.action for x in minibatch])
    rewards = np.array([x.reward for x in minibatch])
    next_states = np.vstack([x.next_state for x in minibatch])
    done = np.array([x.done for x in minibatch])

    Q_predict = agent.get_Q(states)
    Q_target = Q_predict.clone().data.numpy()
    Q_target[np.arange(len(Q_target)), actions] = rewards + gamma * np.max(agent.get_Q(next_states).data.numpy(), axis=1) * ~done
    Q_target = agent._to_variable(Q_target)

    return agent.train(Q_predict, Q_target)


def play_episode(env: gym.Env,
                 agent: Agent,
                 replay_memory: ReplayMemory,
                 eps: float,
                 batch_size: int,
                 render = False) -> int:
    """Play an epsiode and train
    Args:
        env (gym.Env): gym environment (CartPole-v0)
        agent (Agent): agent will train and get action
        replay_memory (ReplayMemory): trajectory is saved here
        eps (float): 𝜺-greedy for exploration
        batch_size (int): batch size
    Returns:
        int: reward earned in this episode
    """
    s = env.reset()
    done = False
    total_reward = 0

    while not done:

        a = agent.get_action(s, eps)
        s2, r, done, info = env.step(a)

        if render == True:
            env.render()

        total_reward += r

        if done:
            r = -1
        replay_memory.push(s, a, r, s2, done)

        if len(replay_memory) > batch_size:

            minibatch = replay_memory.pop(batch_size)
            train_helper(agent, minibatch, FLAGS["gamma"])

        s = s2

    return total_reward


def get_env_dim(env: gym.Env) -> Tuple[int, int]:
    """Returns input_dim & output_dim
    Args:
        env (gym.Env): gym Environment (CartPole-v0)
    Returns:
        int: input_dim
        int: output_dim
    """
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    return input_dim, output_dim


def epsilon_annealing(epsiode: int, max_episode: int, min_eps: float) -> float:
    """Returns 𝜺-greedy
    1.0---|\
          | \
          |  \
    min_e +---+------->
              |
              max_episode
    Args:
        epsiode (int): Current episode (0<= episode)
        max_episode (int): After max episode, 𝜺 will be `min_eps`
        min_eps (float): 𝜺 will never go below this value
    Returns:
        float: 𝜺 value
    """

    slope = (min_eps - 1.0) / max_episode
    return max(slope * epsiode + 1.0, min_eps)

In [ ]:
Transition = namedtuple("Transition",
                        field_names=["state", "action", "reward", "next_state", "done"])


env = CartsPolesEnv()
rewards = np.zeros(FLAGS['n_episode'])
input_dim, output_dim = get_env_dim(env)
agent = Agent(input_dim, output_dim, FLAGS['hidden_dim'])
replay_memory = ReplayMemory(FLAGS['capacity'])

for i in range(n_episode):
    render = False
    eps = epsilon_annealing(i, max_episode, FLAGS['min_eps'])

    if (i % 50 == 0): render = False

    r = play_episode(env, agent, replay_memory, eps,FLAGS['batch_size'],render)
    if (i % 100 == 0): print("[Episode: {:5}] Reward: {:5} 𝜺-greedy: {:5.2f}".format(i + 1, r, eps))

    rewards[i] = r

    # if len(rewards) == rewards.maxlen:
    #     if np.mean(rewards) >= 200:
    #         print("Game cleared in {} games with {}".format(i + 1, np.mean(rewards)))
    #         break

# fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(12,4), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(range(0,FLAGS['n_episode']),rewards)
# ax2.plot(range(0,n_episode),td_error,c='g')
plt.title("DQN Episode Length vs Episode")
# ax2.title.set_text("TD Error Convergence")
# fig.suptitle('DQN Control')
# ax1.set_xscale('log')
# ax2.set_xscale('log')
plt.grid()
# ax2.grid()
plt.show()

[Episode:     1] Reward: 0.7100000000000004 𝜺-greedy:  1.00
[Episode:     2] Reward: 0.7700000000000005 𝜺-greedy:  0.98
[Episode:     3] Reward: 0.7300000000000004 𝜺-greedy:  0.96
[Episode:     4] Reward: 0.48000000000000026 𝜺-greedy:  0.94
[Episode:     5] Reward: 0.6800000000000004 𝜺-greedy:  0.92
[Episode:     6] Reward: 0.7100000000000004 𝜺-greedy:  0.90
[Episode:     7] Reward: 0.6000000000000003 𝜺-greedy:  0.88
[Episode:     8] Reward: 0.6200000000000003 𝜺-greedy:  0.86
[Episode:     9] Reward: 0.5000000000000002 𝜺-greedy:  0.84
[Episode:    10] Reward: 0.6100000000000003 𝜺-greedy:  0.82
[Episode:    11] Reward: 0.5900000000000003 𝜺-greedy:  0.80
[Episode:    12] Reward: 0.6000000000000003 𝜺-greedy:  0.78
[Episode:    13] Reward: 0.6600000000000004 𝜺-greedy:  0.76
[Episode:    14] Reward: 0.5900000000000003 𝜺-greedy:  0.74
[Episode:    15] Reward: 0.6700000000000004 𝜺-greedy:  0.72
[Episode:    16] Reward: 0.7000000000000004 𝜺-greedy:  0.70
[Episode:    17] Reward: 0.720000000000

[Episode:   136] Reward: 0.9000000000000006 𝜺-greedy:  0.01
[Episode:   137] Reward: 0.4000000000000002 𝜺-greedy:  0.01
[Episode:   138] Reward: 0.4000000000000002 𝜺-greedy:  0.01
[Episode:   139] Reward: 0.5200000000000002 𝜺-greedy:  0.01
[Episode:   140] Reward: 0.49000000000000027 𝜺-greedy:  0.01
[Episode:   141] Reward: 0.49000000000000027 𝜺-greedy:  0.01
[Episode:   142] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   143] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   144] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   145] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   146] Reward: 0.5100000000000002 𝜺-greedy:  0.01
[Episode:   147] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   148] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   149] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   150] Reward: 0.5100000000000002 𝜺-greedy:  0.01
[Episode:   151] Reward: 0.37000000000000016 𝜺-greedy:  0.01
[Episode:   152] Reward: 0.3100000000

[Episode:   270] Reward: 0.6400000000000003 𝜺-greedy:  0.01
[Episode:   271] Reward: 0.6300000000000003 𝜺-greedy:  0.01
[Episode:   272] Reward: 0.6100000000000003 𝜺-greedy:  0.01
[Episode:   273] Reward: 0.6100000000000003 𝜺-greedy:  0.01
[Episode:   274] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   275] Reward: 0.6000000000000003 𝜺-greedy:  0.01
[Episode:   276] Reward: 0.6100000000000003 𝜺-greedy:  0.01
[Episode:   277] Reward: 0.6000000000000003 𝜺-greedy:  0.01
[Episode:   278] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   279] Reward: 0.6100000000000003 𝜺-greedy:  0.01
[Episode:   280] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   281] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   282] Reward: 0.6300000000000003 𝜺-greedy:  0.01
[Episode:   283] Reward: 0.6100000000000003 𝜺-greedy:  0.01
[Episode:   284] Reward: 0.6000000000000003 𝜺-greedy:  0.01
[Episode:   285] Reward: 0.6000000000000003 𝜺-greedy:  0.01
[Episode:   286] Reward: 0.6000000000000

[Episode:   405] Reward: 0.5400000000000003 𝜺-greedy:  0.01
[Episode:   406] Reward: 0.46000000000000024 𝜺-greedy:  0.01
[Episode:   407] Reward: 0.47000000000000025 𝜺-greedy:  0.01
[Episode:   408] Reward: 0.47000000000000025 𝜺-greedy:  0.01
[Episode:   409] Reward: 0.6900000000000004 𝜺-greedy:  0.01
[Episode:   410] Reward: 0.6500000000000004 𝜺-greedy:  0.01
[Episode:   411] Reward: 0.6700000000000004 𝜺-greedy:  0.01
[Episode:   412] Reward: 0.5300000000000002 𝜺-greedy:  0.01
[Episode:   413] Reward: 0.45000000000000023 𝜺-greedy:  0.01
[Episode:   414] Reward: 0.3900000000000002 𝜺-greedy:  0.01
[Episode:   415] Reward: 0.4000000000000002 𝜺-greedy:  0.01
[Episode:   416] Reward: 0.4200000000000002 𝜺-greedy:  0.01
[Episode:   417] Reward: 1.420000000000001 𝜺-greedy:  0.01
[Episode:   418] Reward: 0.35000000000000014 𝜺-greedy:  0.01
[Episode:   419] Reward: 0.48000000000000026 𝜺-greedy:  0.01
[Episode:   420] Reward: 0.3300000000000001 𝜺-greedy:  0.01
[Episode:   421] Reward: 0.32000000

[Episode:   539] Reward: 0.6300000000000003 𝜺-greedy:  0.01
[Episode:   540] Reward: 0.38000000000000017 𝜺-greedy:  0.01
[Episode:   541] Reward: 0.6000000000000003 𝜺-greedy:  0.01
[Episode:   542] Reward: 0.5700000000000003 𝜺-greedy:  0.01
[Episode:   543] Reward: 0.36000000000000015 𝜺-greedy:  0.01
[Episode:   544] Reward: 0.45000000000000023 𝜺-greedy:  0.01
[Episode:   545] Reward: 0.5100000000000002 𝜺-greedy:  0.01
[Episode:   546] Reward: 0.47000000000000025 𝜺-greedy:  0.01
[Episode:   547] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   548] Reward: 0.6200000000000003 𝜺-greedy:  0.01
[Episode:   549] Reward: 0.37000000000000016 𝜺-greedy:  0.01
[Episode:   550] Reward: 0.6500000000000004 𝜺-greedy:  0.01
[Episode:   551] Reward: 0.5000000000000002 𝜺-greedy:  0.01
[Episode:   552] Reward: 0.5200000000000002 𝜺-greedy:  0.01
[Episode:   553] Reward: 0.4300000000000002 𝜺-greedy:  0.01
[Episode:   554] Reward: 0.38000000000000017 𝜺-greedy:  0.01
[Episode:   555] Reward: 0.7400000